In [1]:
#Bar Graph Function
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
from scipy.optimize import leastsq
import scipy
import math
from scipy.odr import ODR, Model, RealData
import matplotlib
import scipy.stats

def GaussianFitting(beta, x):
    y_fit = beta[0]*norm.pdf(x, beta[1], beta[2]) 
    return y_fit

#Bar Graph Function
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
from scipy.optimize import leastsq
import scipy
import math
from scipy.odr import ODR, Model, RealData
import matplotlib
import scipy.stats

def GaussianFitting(beta, x):
    y_fit = beta[0]*norm.pdf(x, beta[1], beta[2]) 
    return y_fit

def BarGraph(Array, Bins, Title, X_Label, Y_Label, xPos, yPos, limits = False, Gaussian = True, lowlim = 0, uplim = 0,
            xlog = False, ylog = False):
    
    fig = plt.figure(figsize=(15,6))
    ax = plt.gca()
    font = {'size'   : 20}
    plt.rc('font', **font)

    if (xlog == True):
        plt.xscale('log')
        binArray = np.logspace(np.log10(np.min(Array)),np.log10(np.max(Array)), Bins)
        x_axis = np.logspace(np.log10(np.min(Array)), np.log10(np.max(Array)), 1000)
    else:
        binArray = np.linspace(np.min(Array), np.max(Array), Bins)
        x_axis = np.linspace(np.min(Array), np.max(Array), 1000)
        
    if (ylog == True):
        plt.yscale('log')
        
    n, bins, patches = plt.hist(Array, binArray, density=False, facecolor='midnightblue', alpha=0.75)
    plt.xlabel(X_Label)
    plt.ylabel(Y_Label)
    plt.title(Title)
    plt.grid(True, color='lightgrey')
    
    #ax.set_facecolor('xkcd:really light blue')
    mean = np.average(Array)
    sd = np.std(Array)
    
    if (limits == True):
        plt.xlim(lowlim, uplim)
        
    if (Gaussian == True): #Fitting Gaussian to Histogram
        
        #Centering bins coordinates
        Bin_CenterVal = (bins[1] - bins[0])/2
        bins_Centered = np.zeros(Bins)
        OneBinLess = (Bins-1)
        i = 0
        while i < OneBinLess:
            bins_Centered[i] = bins[i] + Bin_CenterVal
            i+=1
            
        model = Model(GaussianFitting)
        data = RealData(bins_Centered, n)
        guess1 = np.max(n) #FIRST GUESS of beta[0]
        guess2 = mean      #SECOND GUESS of beta[1] #ADD MORE GUESSES IF NEEDED
        guess3 = sd
        odr = ODR(data, model, [guess1, guess2, guess3]
        #, maxit = 50*10**3 #option for maximum iterations if there are many parameters
         )
        odr.set_job(fit_type = 0) #0 Explicit ODR, 1 Implicit ODR, 2 Ordinary Least Squares
        output = odr.run()
        beta = output.beta
        beta_error = output.sd_beta
        
        fit = GaussianFitting(beta, x_axis)
        
        
        plt.plot(x_axis, fit, ls = "--")
        
        fig.text(xPos, yPos, 
             "Median = %.3e\n"
             "$\mu$ = %0.3e     $\sigma$ = %0.3e\n" 
             "Fitting:\n"
             "Amplitude = %0.3e $\pm$ %0.3e\n"   
             "$\mu$ = %0.3e $\pm$ %0.3e\n"
             "$\sigma$ = %0.3e $\pm$ %0.3e" % 
             (statistics.median(Array), mean, sd, beta[0], beta_error[0], 
              beta[1], beta_error[1], beta[2], beta_error[2])
             ,bbox={'facecolor': 'cornsilk'}, fontsize=16)   
    else:
        fig.text(xPos, yPos, 
             "Median = %.3e\n"
             "$\mu$ = %0.3e\n" 
             "$\sigma$ = %0.3e\n" % 
             (statistics.median(Array), mean, sd)
             ,bbox={'facecolor': 'cornsilk'}, fontsize=16)  
        
    plt.show()
    
    return